<a href="https://colab.research.google.com/github/tejpal123456789/NLP-with-pytorch-and-pytorch-text/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 26.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.0
    Uninstalling torchtext-0.13.0:
      Successfully uninstalled torchtext-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0+cu113 requires torch==1.12.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.9.0 which is incompatible.


In [2]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 29.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import spacy
spacy_en = spacy.load('en_core_web_sm')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [4]:
import torch
import torchtext

SEED = 1234
from torchtext.legacy.datasets import IMDB
from torchtext.legacy.data import Field, BucketIterator,LabelField

import spacy
import numpy as np

import random
import math
import time
print(torch.__version__)
print(torchtext.__version__)

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = Field(tokenize = tokenize_en, 
            
            lower = True)

LABEL = LabelField(dtype = torch.float)

1.9.0+cu102
0.10.0


In [5]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.3MB/s]


In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [7]:
print(vars(train_data.examples[0]))

{'text': ['rohmer', 'strays', 'from', 'his', 'usual', 'portraits', 'of', 'french', 'middle', 'class', 'to', 'tell', 'this', 'costume', 'drama', 'about', 'the', 'difficulties', 'of', 'an', 'aristocrat', 'lady', 'during', 'the', 'french', 'revolution', '.', 'what', "'s", 'more', 'attractive', 'about', '"', 'la', 'anglaise', '...', '"', '(', 'apart', 'from', 'the', 'story', 'itself', ')', 'is', 'the', 'fabulous', 'aesthetics', 'that', 'rohmer', 'has', 'achieved', '.', 'the', 'images', 'have', 'been', 'digitally', 'decorated', 'too', 'make', 'them', 'look', 'like', 'baroque', 'pictures', '.', 'in', 'some', 'moments', 'you', 'ca', "n't", 'really', 'say', 'whether', 'your', 'watching', 'a', 'movie', 'or', 'a', 'series', 'of', 'pictures', 'in', 'louvre', 'musseum', '.', 'every', 'shot', 'is', 'like', 'a', 'piece', 'of', 'art.<br', '/><br', '/>*my', 'rate', ':', '7.5/10', '<', 'br', '/><br', '/>----------------<br', '/><br', '/>--------------', '-<br', '/><br', '/>------------<br', '/><br', '/

In [8]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [10]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)


In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231205), (',', 194115), ('.', 166392), ('and', 114260), ('a', 113985), ('of', 102182), ('to', 95387), ('is', 77939), ('it', 66099), ('in', 65364), ('i', 58484), ('that', 51743), ('this', 51640), ('"', 43933), ("'s", 43930), ('-', 37651), ('/><br', 35707), ('was', 35342), ('as', 32575), ('for', 30945)]


In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [16]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [56]:
for idx,batch in enumerate(train_iterator):
  print(batch.text[0])
  if idx>0:
    break


tensor([   31,    14,    21,  6693,    12,    15,    12,  4449,     5,    12,
            6,    12,    12,     2,    12,     2,  2757,    12,   249,    45,
         1213, 16084,   114,    14,   994,    93,     2,    12,   390,    12,
           12,    12,   380,    12,   301,    12,    12,    12,  2484,  6970,
          466,   301,    12,    12,     6,  1261,     2,    57,    14,    15,
         1897,   601,  1065,   115,   272,   305,   112,    42,    54,   635,
            2,    72,    15,    10], device='cuda:0')
tensor([ 3341,  1492,   102,    14,    15,    64,    12,   328,     6,    15,
        11324,  3845,    14,   287,    12,    57,    14,    12,    10,    14,
           12,    75,     2,    20,   514,    31,     2,    11,    31,    14,
          176,    50,   842,    20,  2941,     2,  7672,  3071,    12,     2,
           11,    12,    72,    10,   106,    14,  1280,     0,  3582,   314,
           12,    14,    14,   635,    14,   114,  1809,    14,     2, 10930,
          

In [17]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

We now create an instance of our RNN class.

The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size.

The embedding dimension is the size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

The hidden dimension is the size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.



In [18]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [20]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [21]:
criterion = nn.BCEWithLogitsLoss()

In [22]:
model = model.to(device)
criterion = criterion.to(device)

In [23]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [42]:
from tqdm import tqdm
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for idx,batch in tqdm(enumerate(iterator),total=len(iterator)):
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [46]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|██████████| 274/274 [00:17<00:00, 15.99it/s]


Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.693 | Train Acc: 50.48%
	 Val. Loss: 0.698 |  Val. Acc: 48.86%


100%|██████████| 274/274 [00:15<00:00, 17.77it/s]


Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 49.97%
	 Val. Loss: 0.698 |  Val. Acc: 49.76%


100%|██████████| 274/274 [00:15<00:00, 17.29it/s]


Epoch: 03 | Epoch Time: 0m 17s
	Train Loss: 0.693 | Train Acc: 50.30%
	 Val. Loss: 0.697 |  Val. Acc: 49.03%


100%|██████████| 274/274 [00:15<00:00, 17.52it/s]


Epoch: 04 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 50.14%
	 Val. Loss: 0.698 |  Val. Acc: 48.82%


100%|██████████| 274/274 [00:18<00:00, 15.05it/s]


Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 0.693 | Train Acc: 49.88%
	 Val. Loss: 0.698 |  Val. Acc: 48.91%
